# Generate Data

In [1]:
import numpy as np
import pandas as pd
from mlchecks import Dataset, validate_dataset, validate_dataset_or_dataframe
from mlchecks.checks.integrity.rare_format_detection import RareFormatDetection



# desired_width=320
# pd.set_option('display.width', desired_width)
# pd.set_option('display.max_columns', None)

np.random.seed(42)

df = pd.DataFrame(np.random.choice(a=['BIG', 'STILL_BIG'], size=(200,3)), columns=['x1', 'x2', 'x3'])
df = df.append({'x1': 'bla', 'x2': 'BIG', 'x3': 1}, ignore_index=True)
df = df.append({'x1': 'bla', 'x2': 'BIG', 'x3': 1}, ignore_index=True)
df = df.append({'x1': 'bla2', 'x2': 'BIG', 'x3': 2}, ignore_index=True)

# Run Check

In [2]:
ds = Dataset(df)
check = RareFormatDetection()

check.run(dataset=ds)


c:\users\nirhu\documents\deepchecks\mlchecks\mlchecks\base\dataset.py:57: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._features = [x for x in df.columns if x not in {label, index, date}]
c:\users\nirhu\documents\deepchecks\mlchecks\mlchecks\base\dataset.py:67: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._cat_features = self.infer_categorical_features()


,digits only format (ignoring letters),digits and letters format,digits and letters format (case sensitive),any sequence format
ratio of rare patterns to common patterns,0.95%,0.50%,1.50%,0.50%
common formats,[_],"[XXXXX_XXX, XXX]","[XXXXX_XXX, XXX]","[SEQ_SEQ, SEQ]"
examples for values in common formats,[STILL_BIG],"[STILL_BIG, BIG]","[STILL_BIG, BIG]","[STILL_BIG, BIG]"
values in rare formats,[bla2],[bla2],"[bla, bla2]",[bla2]
,digits only format (ignoring letters),digits and letters format,digits and letters format (case sensitive),any sequence format
ratio of rare patterns to common patterns,2.75%,1.50%,1.50%,1.50%
common formats,[_],"[XXXXX_XXX, XXX]","[XXXXX_XXX, XXX]","[SEQ_SEQ, SEQ]"
examples for values in common formats,[STILL_BIG],"[STILL_BIG, BIG]","[STILL_BIG, BIG]","[STILL_BIG, BIG]"
values in rare formats,"[1, 2]","[1, 2]","[1, 2]","[1, 2]"
